In [6]:
search_for = 110
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.009310007095336914
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 389918155
type: [1, 1, 1, 1, 110] 110
cases of this type: 146410000
10000 50.018165903659366
20000 44.73468361347172
30000 50.56150585579468
40000 113.35028799258708
50000 222.39549475947615
60000 252.07959577656422
70000 255.6419167698633
80000 274.13141098063545
90000 295.5540199748349
100000 272.4793822358885
110000 275.75943445112796
120000 299.5946644707033
130000 278.97646951462866
140000 280.2001413630786
150000 288.63006113536517
160000 273.35689240039545
170000 285.3169624080537
180000 289.4213234911714
190000 267.9532419428007
200000 277.787776339516
210000 288.11454233243404
220000 268.2205375121927
230000 277.4334587274798
240000 273.4052371251398
250000 279.6358350553531
260000 280.83654404507763
270000 272.2884587017576
280000 286.5954185027648
290000 280.081906546084
300000 270.5015878658218
310000 279.1168203901

3080000 271.3948876309597
3090000 271.6888983375498
3100000 267.79589505649267
3110000 260.56893299326146
3120000 267.6221089808107
3130000 257.3500163963811
3140000 271.51001491944425
3150000 275.245317494002
3160000 257.3272383183912
3170000 279.6377559104793
3180000 271.1126371316368
3190000 255.98680812273594
3200000 283.8677503515674
3210000 271.58529816526084
3220000 259.25255533179075
3230000 287.6964884026797
3240000 258.4197381185878
3250000 251.07549431991308
3260000 298.126415764599
3270000 258.432122347501
3280000 255.58342225200207
3290000 328.06891236864783
3300000 327.3039307444217
3310000 146.00628846335022
3320000 35.26227100776381
3330000 35.28134949607558
3340000 35.084197075523335
3350000 34.80978723047048
3360000 34.98266197714574
3370000 232.35955687971293
3380000 357.45046390441365
3390000 289.2110835088334
3400000 285.1893452348894
3410000 272.81245979235183
3420000 257.05609040414095
3430000 278.7484070931879
3440000 260.33544446509256
3450000 249.8953414921432

6190000 250.04874115238758
6200000 254.85358648775565
6210000 249.64673997567877
6220000 252.58653969150325
6230000 272.2406487354406
6240000 255.43242823955416
6250000 253.9214109244463
6260000 252.8961410772878
6270000 267.57694367009066
6280000 254.8567328154328
6290000 264.2292991126371
6300000 269.799445868055
6310000 262.73181753241926
6320000 258.7800095869219
6330000 290.15833118041814
6340000 272.58108736814233
6350000 264.88112699385556
6360000 289.02689703941843
6370000 277.5171750626835
6380000 268.9381462588656
6390000 265.3085666484806
6400000 272.7567931680426
6410000 261.4068419980655
6420000 262.32395712369447
6430000 264.5622484894636
6440000 262.6727060055483
6450000 281.2888844686596
6460000 287.33837693995775
6470000 263.3562851212922
6480000 262.73176634218635
6490000 273.7818645527024
6500000 251.0877064837039
6510000 280.90018934591217
6520000 274.72650711124095
6530000 249.4066570337593
6540000 269.0978617625533
6550000 260.66785307194107
6560000 247.7565118829

9300000 278.74836962960575
9310000 306.99191897578953
9320000 264.94386949648316
9330000 267.52049520275403
9340000 281.2551138163742
9350000 243.14507810448916
9360000 259.9104687320958
9370000 312.0442195323138
9380000 249.9105589874902
9390000 68.2699819413498
9400000 68.27266602467537
9410000 68.78368546979027
9420000 69.85944180809736
9430000 70.67041367844453
9440000 108.62488416687349
9450000 336.9689157389858
9460000 296.5524072259719
9470000 251.7030053450903
9480000 274.67321720730985
9490000 267.2921659185365
9500000 253.05692365093316
9510000 272.4480714660512
9520000 272.9396860960054
9530000 254.38987958047002
9540000 259.9661890600191
9550000 241.5184058980643
9560000 249.04015545840292
9570000 264.3285365018306
9580000 259.8469324314511
9590000 248.14314707539128
9600000 255.6378257974612
9610000 273.7790623784414
9620000 274.0760316196965
9630000 268.2002909162239
9640000 273.0214039260967
9650000 262.44113384407393
9660000 264.2648589407356
9670000 269.4303438288161
9

12320000 266.069890332466
12330000 248.80829045115638
12340000 241.6463554189824
12350000 257.83722903585044
12360000 273.3616773750387
12370000 256.56041785081567
12380000 262.42645506192014
12390000 286.0161788837042
12400000 254.73103397999745
12410000 265.4677032626136
12420000 258.2580626904965
12430000 266.55043386223645
12440000 265.62737253916725
12450000 269.8434657788945
12460000 254.76802288089573
12470000 252.54778572271323
12480000 257.0534765362953
12490000 248.95147340358994
12500000 260.5605075158389
12510000 254.70342900455472
12520000 236.07099027652862
12530000 250.0648523057595
12540000 268.44872929790006
12550000 240.6089352080178
12560000 264.3487012085727
12570000 248.8672895221978
12580000 223.0732161027206
12590000 234.64158084220435
12600000 239.49123976499192
12610000 240.6025617243444
12620000 309.16407493084654
12630000 45.89106240169296
12640000 34.6797871661523
12650000 34.25062178098093
12660000 34.42470510544008
12670000 34.17570741365622
12680000 34.17

...

377270000 2.269531595454481
377280000 2.1613766551808515
377290000 2.29390915815562
377300000 2.2587153468953236
377310000 2.194633245608277
377320000 2.431181831895411
377330000 1.2624110475551087
377340000 1.672201831007699
377350000 1.021740315413475
377360000 2.243524808080064
377370000 2.194469479184184
377380000 3.016491140297651
377390000 3.612225573389166
377400000 2.227869958072172
377410000 2.1834442940211294
377420000 0.942400718745258
377430000 1.8549328165723549
377440000 2.25611919042329
377450000 3.4466804748838147
377460000 4.365170981516772
377470000 3.17676579993099
377480000 2.883356095593373
377490000 2.506182852332526
377500000 2.1583159446671605
377510000 1.842072749818994
377520000 1.0250634464713269
377530000 2.24719705442518
377540000 3.8794669556817745
377550000 2.5872428768384457
377560000 2.2091529389158886
377570000 2.183095242410567
377580000 1.3230029685704245
377590000 0.5830505482110381
377600000 4.071983240685198
377610000 2.167270818175011
377620000 2

380140000 1.7113121975736816
380150000 1.7361795558128423
380160000 1.7335702416935896
380170000 2.772000574029088
380180000 1.6805106037912767
380190000 1.559339135524432
380200000 1.5982767664673925
380210000 2.3026302357822326
380220000 1.3162895523526603
380230000 0.8169085866850614
380240000 1.9446202047098013
380250000 0.7920243618186315
380260000 1.7441103754051523
380270000 0.9473328794690966
380280000 1.8109788785531455
380290000 1.7190062471610308
380300000 1.5825430634897948
380310000 1.5966011253512569
380320000 1.6752156006153425
380330000 1.9131432884069286
380340000 1.7890225937478739
380350000 1.6493570097752415
380360000 1.4966603263282114
380370000 1.6469789054164292
380380000 1.6921408946853878
380390000 1.2335205971116823
380400000 0.8044477542287443
380410000 0.9094339060075084
380420000 0.8024264734800657
380430000 1.0943205126096143
380440000 1.5491789121239383
380450000 2.6413412000798187
380460000 1.5784270015529789
380470000 1.5310704720133543
380480000 1.5882

382980000 1.2153463948972358
382990000 1.2283579552060364
383000000 1.2869911357775992
383010000 1.2335198731151222
383020000 1.255645586932699
383030000 1.2275859120411012
383040000 0.9183792763038146
383050000 0.7131544190992911
383060000 1.3154289087678328
383070000 0.7607231549369627
383080000 0.38978385587006803
383090000 1.2687052687815163
383100000 0.8899104180659518
383110000 0.5719658505654335
383120000 1.1842956783836747
383130000 1.4183688682677018
383140000 1.719671733402411
383150000 1.345554312614302
383160000 1.4417290040673483
383170000 2.28109023448348
383180000 1.261955042915576
383190000 1.1901650539462103
383200000 1.4920287209774057
383210000 0.845989488904145
383220000 1.0344719678713217
383230000 1.1407711096404989
383240000 0.44272736523744133
383250000 0.8516428789681859
383260000 1.2298086680406333
383270000 0.8196370002987649
383280000 0.5646511388934321
383290000 1.1299539468662938
383300000 1.1588709500198564
383310000 1.3716888025458984
383320000 1.3744156

385820000 0.3607886777559916
385830000 0.8032445770015318
385840000 0.9822504981735348
385850000 0.7919184004335602
385860000 0.8262323865921629
385870000 0.8375159792035818
385880000 0.7522722242709663
385890000 1.1660760079863999
385900000 0.4075294970901807
385910000 0.2957254461255007
385920000 0.7784085115991367
385930000 1.3835781024162968
385940000 0.8111456442724664
385950000 0.8477869131257799
385960000 1.1542296991541983
385970000 0.9436795751107733
385980000 0.6863410837745335
best so far: 0
type: [2, 1, 1, 5, 11] 110
cases of this type: 1830125
385990000 0.6888291798922419
386000000 0.6032648960681095
386010000 0.29169628715773427
386020000 0.4993919587008159
386030000 0.3193697633911172
386040000 0.3319251156888737
386050000 0.3446084067571163
386060000 0.31000319500403273
386070000 0.2808331525596314
386080000 0.17738866222331923
386090000 0.17620286894265147
386100000 0.21010346658468249
386110000 0.34346728498677415
386120000 0.26917155520170927
386130000 0.323641748956

388550000 0.09940284253150225
388560000 0.08511767734742827
388570000 0.07953125496089458
388580000 0.08673171610491144
388590000 0.09393524525781473
388600000 0.0930023712549607
388610000 0.1510666912392113
388620000 0.11010425283445252
388630000 0.09338669519394635
388640000 0.08806286789092753
best so far: 0
type: [2, 1, 1, 55, 1] 110
cases of this type: 166375
388650000 0.07517964429977866
388660000 0.04497757267564535
388670000 0.04468692487309377
388680000 0.04428422755867243
388690000 0.043852742615838845
388700000 0.04333608087672128
388710000 0.0714123686605692
388720000 0.04337669471025467
388730000 0.0439288359480434
388740000 0.043897075187034076
388750000 0.04363445621699095
388760000 0.04196321683896913
388770000 0.04171754057218631
388780000 0.04135654064496359
388790000 0.040458905637131795
388800000 0.03969049987461832
388810000 0.03917710089524587
best so far: 0
type: [2, 1, 5, 1, 11] 110
cases of this type: 366025
388820000 0.09744716681321462
388830000 0.08818628452